# 単語の意味をベクトル化（Doc2Vec）

# ライブラリのインストール

In [1]:
import zipfile
import os.path
import os
import urllib.request as req
import MeCab
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from line_notify import line_notice

# Mecabの初期化

In [2]:
mecab = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
mecab.parse('')

'EOS\n'

# 学習対象とする青空文庫の作品リストの作成

In [3]:
list = [
    {"auther":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"}]
    },
    {"auther":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"}]
    },
    {"auther":{
        "name":"ポー エドガー・アラン",
        "url":"https://www.aozora.gr.jp/cards/000094/files/"}, 
     "books":[
        {"name":"ウィリアム・ウィルスン","zipname":"2523_ruby_19896.zip"},
        {"name":"落穴と振子","zipname":"1871_ruby_17551.zip"},
        {"name":"黒猫","zipname":"530_ruby_20931.zip"},
        {"name":"群集の人","zipname":"56535_ruby_69925.zip"},
        {"name":"沈黙","zipname":"56537_ruby_70425.zip"}]
    },
    {"auther":{
        "name":"紫式部",
        "url":"https://www.aozora.gr.jp/cards/000052/files/"}, 
     "books":[
        {"name":"源氏物語 01 桐壺","zipname":"5016_ruby_9746.zip"},
        {"name":"源氏物語 02 帚木","zipname":"5017_ruby_9752.zip"},
        {"name":"源氏物語 03 空蝉","zipname":"5018_ruby_9754.zip"},
        {"name":"源氏物語 04 夕顔","zipname":"5019_ruby_9761.zip"},
        {"name":"源氏物語 05 若紫","zipname":"5020_ruby_11253.zip"}]
    }
]

In [4]:
list

[{'auther': {'name': '宮澤 賢治',
   'url': 'https://www.aozora.gr.jp/cards/000081/files/'},
  'books': [{'name': '銀河鉄道の夜', 'zipname': '43737_ruby_19028.zip'},
   {'name': '注文の多い料理店', 'zipname': '1927_ruby_17835.zip'},
   {'name': 'セロ弾きのゴーシュ', 'zipname': '470_ruby_3987.zip'},
   {'name': 'やまなし', 'zipname': '46605_ruby_29758.zip'},
   {'name': 'どんぐりと山猫', 'zipname': '43752_ruby_17595.zip'}]},
 {'auther': {'name': '芥川 竜之介',
   'url': 'https://www.aozora.gr.jp/cards/000879/files/'},
  'books': [{'name': '羅生門', 'zipname': '127_ruby_150.zip'},
   {'name': '鼻', 'zipname': '42_ruby_154.zip'},
   {'name': '河童', 'zipname': '69_ruby_1321.zip'},
   {'name': '歯車', 'zipname': '42377_ruby_34744.zip'},
   {'name': '老年', 'zipname': '131_ruby_241.zip'}]},
 {'auther': {'name': 'ポー エドガー・アラン',
   'url': 'https://www.aozora.gr.jp/cards/000094/files/'},
  'books': [{'name': 'ウィリアム・ウィルスン', 'zipname': '2523_ruby_19896.zip'},
   {'name': '落穴と振子', 'zipname': '1871_ruby_17551.zip'},
   {'name': '黒猫', 'zipname': '530_

# 関数の定義

In [5]:
# 作品リストを取得してループ処理に渡す
def book_list():
    for novelist in list:
        auther = novelist['auther']
        for book in novelist['books']:
            yield auther, book # for文で呼び出した時に、順々に値を返すように yield を使用

In [6]:
# zipファイルを開き、中の文章を取得する
def read_book(auther, book, data_dir=''):
    zipname = book['zipname']
    # zipファイルがなければ取得する
    if data_dir != '':
        data_dir = data_dir + '/'
    if not os.path.exists(data_dir + zipname):
        data = req.urlopen(auther['url'] + zipname).read()
        with open(data_dir + zipname, mode="wb") as f:
            f.write(data)
    # zipファイルを開く
    with zipfile.ZipFile(data_dir + zipname, 'r') as zf:
        # zipファイルに含まれるファイルを開く。今回はzipは1つのテキストファイルのみを含む
        for filename in zf.namelist():
            with zf.open(filename, 'r') as f:
                return f.read().decode('shift-jis')

In [7]:
# 引数のテキストを分かち書きして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

# 学習データの前処理

In [8]:
# 作品リストをDoc2Vecが読めるTaggedDocument形式にし、配列に追加する
documents = []
# 作品リストをループで回す
for auther, book in book_list():
    # 作品の文字列を取得
    words = read_book(auther, book, 'data')
    # 作品の文字列を分かち書きに
    wakati_words = split_words(words)
    # TaggedDocumentの作成　文書=分かち書きにした作品　タグ=作者:作品名
    document = TaggedDocument(wakati_words, [auther["name"] + ":" + book["name"]])
    documents.append(document)

In [27]:
# words
documents[0][0][55:65]

['みなさん', 'ふう', '川', '言', 'い', 'われ', '乳', '流る', 'れる', 'あと']

In [11]:
# tags
documents[0][1]

['宮澤 賢治:銀河鉄道の夜']

# モデルの作成・保存

In [13]:
# TaggedDocumentの配列を使ってDoc2Vecの学習モデルを作成
# dm：Doc2Vecで使うアルゴリズムを選択。（1=dmpw, 0=DBOW）※doc2vecではdmpwの方が計算時間はかかるものの精度が高い ※word2vec（Skip-gram, CBOW：Skip-gramの方が精度が高い）
# size：ベクトルの次元を設定。（Doc2Vecでは基本的に300が良いとされています）
# window：学習する単語の前後数。（dmpwでは5が良いとされている）
# min_count：最低何回出てきた文字列を対象とするかを設定。（今回は作家ごとに独特の言い回しなどがあると考えられるため、1回でも出てきた文字列を対象にしている）
model = models.Doc2Vec(documents, dm=0, vector_size=300, window=15, min_count=1)

In [14]:
os.system("osascript -e 'display notification \"Learning finished !!\"'")

0

In [15]:
#Doc2Vecの学習モデルを保存
model.save('model/aozora.model')

In [16]:
line_notice().send_messages('aozora modelの学習が終了しました!!👍')

Messege : 
aozora modelの学習が終了しました!!👍
Response : 成功
